## 나의 팔꿈치 Pose로 로봇암 제어하기

* 앞서 팔꿈치 각도를 시리얼 통신을 이용하여 아두이노에 보냈다.
* 아두이노 측은 해당 각도만큼 서보 모터를 동작시켜 각도를 재현한다.
* 하지만 서보모터가 동작하는데 시간이 걸리고, 서보모터가 동작을 완수하는 동안
* 파이썬에서는 수차례 송신하면서 시리얼에 메시지가 쌓이게되고 각도값이 씹히게 된다.
* 즉, 파이썬과 아두이노간의 동기화 문제가 발생하게 된 것이다.
* 이를 해결하기 위해서
* 파이썬과 아두이노간에 메시지 송수신 체계를 세우고 구현해본다.
    1. 파이썬에서 각도를 보낸다.
    2. 아두이노에서는 각도대로 서보모터를 동작시킨다.
    3. 서보모터가 동작 완료하면 파이썬에게 응답을 보낸다.
    4. 아두이노로부터 동작완료했다는 응답을 받을때에만 새로운 각도를 보낸다.
    

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import threading
import time

isPossibleSendToArd = True # 아두이노에 메시지 보낼 수 있는가?

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 아두이노와의 시리얼 통신을 설정합니다.
arduino = serial.Serial('COM7', 115200, timeout=0.1)  # 포트와 보드레이트를 맞게 설정

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# 아두이노로부터 응답 메시지 기다림
def read_arduino():
    global isPossibleSendToArd

    try:
    
        while True:
            if arduino.in_waiting > 0:
                message = arduino.readline().decode().strip()   # strip() : 읽어온 문자열의 양쪽 공백 제거
                print(f"received message : {message}")

                if message == 'Done':
                    isPossibleSendToArd = True
                elif message == 'Ready':
                    isPossibleSendToArd = True
                else:
                    isPossibleSendToArd = False

            print(f'can send: {isPossibleSendToArd}')

    except Exception as e:
        print(f"Error in read_arduino: {e}")
    finally:
        arduino.close()

# 쓰레드 생성 및 시작
arduino_thread = threading.Thread(target=read_arduino)
arduino_thread.start()
    

while cap.isOpened():
    ret, frame = cap.read()
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
        landmarks = results.pose_landmarks.landmark

        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        shoulder_pos = tuple(np.multiply(shoulder, [640, 480]).astype(int))
        elbow_pos = tuple(np.multiply(elbow, [640, 480]).astype(int))
        wrist_pos = tuple(np.multiply(wrist, [640, 480]).astype(int))

        cv2.circle(image, shoulder_pos, 10, (255,0,0), -1)
        cv2.circle(image, elbow_pos, 10, (0,255,0), -1)
        cv2.circle(image, wrist_pos, 10, (0,0,255), -1)             
        cv2.line(image, shoulder_pos, elbow_pos,   (255,255,0), 3)
        cv2.line(image, elbow_pos, wrist_pos, (0,255,255), 3)
        # 각도 계산하기
        angle = calculate_angle(shoulder, elbow, wrist)

        # 각도 표시하기
        cv2.putText(image, f'Angle: {round(angle, 2)}', tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)


        cv2.putText(image, f'Send: {isPossibleSendToArd}', 
        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)               
        
        # 보낼 수 있는 상황일 경우, 아두이노에 각도 송신하기
        if isPossibleSendToArd:
            arduino.write(f'{int(angle)}\n'.encode())
            isPossibleSendToArd=False

        
    except Exception as e:
        print(f"Error in main loop: {e}")


    cv2.imshow('elbow angle', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break



## 종료되는 순서 주의할 것
arduino.close()
cap.release()
cv2.destroyAllWindows()

# 쓰레드가 종료될 때까지 기다립니다.
arduino_thread.join()